# Регрессия. Часть 1

## Цели работы

1. **Импорт библиотек и загрузка датасета:**
   - Импортировать необходимые библиотеки для работы с данными и моделями машинного обучения.
   - Загрузить и предварительно ознакомиться с данными, предназначенными для анализа и построения модели регрессии.

2. **Выбор лучшей степени полиномиальной регрессии:**
   - Применить метод полиномиальной регрессии для моделирования нелинейных зависимостей.
   - Использовать Grid Search для определения оптимальной степени полинома, обеспечивающей наилучший r2-score.
   - Обучить модель с лучшей степенью полинома и провести предсказание.

3. **Реализация и валидация линейной регрессии собственной реализации:**
   - Реализовать собственный класс линейной регрессии, содержащий методы для обучения модели и предсказания.
   - Обучить модель на тренировочных данных и оценить её качество с помощью метрики r2-score.

4. **Валидация линейной регрессии для датасета California Housing:**
   - Разделить данные на тренировочный и валидационный наборы.
   - Обучить модель линейной регрессии на тренировочных данных и сделать предсказания на валидационном наборе.
   - Оценить качество модели, используя r2-score.
   - Провести Z-преобразование данных и повторить оценку модели, сравнив результаты до и после масштабирования.

Цели данного документа направлены на углубление понимания и практическое применение методов регрессии, а также на изучение способов валидации и оценки качества моделей на примере реальных данных.


## Этап 1. Импорт библиотек и загрузка датасета

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

In [ ]:
data = pd.read_csv("non_linear.csv", delimiter=',')
data.head()

,x_train,y_train
0,0.138368,0.838812
1,0.157237,0.889313
2,0.188684,1.430040
3,0.685553,1.717309
4,0.874237,2.032588


## Этап 2. Выбор лучшей степени полиномиальной регрессии

В рамках данного этапа была подчеркнута важность метрик качества для сравнения различных моделей между собой. В контексте задачи полиномиальной регрессии были выполнены следующие действия:

- Взяты все степени полинома от 1 до 10 по порядку, без пропусков.
- Определена степень полинома, которая обеспечивает наилучший r2-score.
- Реализован код, выводящий наиболее подходящую степень полинома и соответствующий ей r2-score.

Этот процесс, известный как Grid Search, способствовал нахождению оптимальных параметров для модели. После определения лучшей степени полинома, модель была обучена и использована для выполнения предсказаний.


In [3]:
X = data["x_train"].values.reshape(-1, 1)
y = data["y_train"].values.reshape(-1, 1)

best_score = 0
best_degree = 0

In [4]:
for i in range(1, 11):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(X)
    model = LinearRegression()
    model.fit(X_poly, y)

    y_predicted = model.predict(X_poly)
    score = r2_score(y, y_predicted)

    if score > best_score:
        best_score = score
        best_degree = i

print(f"Лучшая степень полинома: {best_degree}")
print(f"Её скор: {best_score}")

Лучшая степень полинома: 10
Её скор: 0.9091133831293501


## Этап 3. Линейная регрессия библиотечной реализации

В этой части работы был разработан и реализован класс для обучения модели линейной регрессии, обладающий следующими возможностями:

- **Метод `.fit(X, y)`:** 
  - Этот метод принимает на вход массив фичей `X` и массив таргетов `y`, после чего обучает коэффициенты регрессии. Процесс обучения основан на аналитическом вычислении коэффициентов, что гарантирует точность и эффективность в получении результатов.

- **Метод `.predict(X)`:**
  - Метод `.predict(X)` используется для предсказания значений. Он принимает массив фичей `X` и возвращает массив предсказаний `y`, основываясь на ранее обученных коэффициентах регрессии.

Реализация этого класса предоставила глубокое понимание механизмов работы линейной регрессии и позволила увидеть, как процесс обучения и предсказания реализуется "внутри" библиотечной реализации. Это задание стало ключевым для осмысления и усвоения концепций линейной регрессии на практике.


In [5]:
class CustomLinearReg:
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None
    
    def fit(self, X, y):
        X_bias = np.c_[np.ones((len(X), 1)), X]
        theta = np.linalg.inv(X_bias.T.dot(X_bias)).dot(X_bias.T).dot(y)
        self.intercept_ = theta[0]
        self.coef_ = theta[1:]
    
    def predict(self, X):
        X_b = np.c_[np.ones((len(X), 1)), X]
        y_predicted = X_b.dot(np.concatenate([[self.intercept_], self.coef_]))
        return y_predicted

In [6]:
reg = CustomLinearReg()
reg.fit(X, y)

print('r2-score: ', r2_score(y, y_predicted))

r2-score:  0.9091133831293501


## Этап 4. Валидация линейной регрессии для датасета Catch California Housing

В рамках этого этапа был осуществлён процесс валидации модели линейной регрессии на данных о домах из Калифорнии. Этапы, которые были выполнены:

1. **Разделение датасета:**
   - Датасет был разделён на две части: тренировочную (`train`) с 80% данных и валидационную (`valid`) с 20% данных. Для разделения использовалась функция `train_test_split` из библиотеки `sklearn`.

2. **Обучение модели:**
   - Модель линейной регрессии была обучена исключительно на тренировочном наборе данных (`train`), что обеспечивает чистоту эксперимента и позволяет оценить способность модели к обобщению на новых данных.

3. **Построение предсказаний на валидационном наборе:**
   - На основе обученной модели были построены предсказания для валидационного набора данных (`valid`).

4. **Расчёт метрики r2 на валидационном наборе:**
   - Для оценки качества модели на валидационном наборе был рассчитан `r2 score`, который представляет собой меру объясняемой моделью дисперсии зависимой переменной.

Далее, было проведено Z-преобразование обоих наборов данных (тренировочного и валидационного), после чего шаги 2-4 были повторены. Это позволило оценить влияние масштабирования признаков на качество модели, выраженное через `r2 score`. Изменение метрики `r2` после Z-преобразования предоставило дополнительные инсайты о структуре и свойствах данных.

Этот процесс стал ключевым для понимания механизмов валидации моделей на отложенной выборке и влияния предобработки данных на результаты моделирования.


In [7]:
california = fetch_california_housing()

X_train, X_valid, y_train, y_valid = train_test_split(
    california.data, california.target, test_size=0.2, random_state=42
)

In [8]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [9]:
y_predicted = lr.predict(X_valid)

In [10]:
r2 = r2_score(y_valid, y_predicted)
print('r2-score:', r2)

r2-score: 0.5757877060324521


In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

y_predicted_scaled = lr.predict(X_valid_scaled)
r2_scaled = r2_score(y_valid, y_predicted_scaled)

print("r2 score с масштабированием:", r2_scaled)

r2 score с масштабированием: 0.575787706032451


После Z-преобразования r2 score совсем не изменился. Скорее всего, набор данных был уже почищен, а трансформация была не нужна.